In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !cd /content/drive/MyDrive/SafeAI/BioID-FaceDatabase-V1.2.zip
# !unzip /content/drive/MyDrive/SafeAI_Final/SafeAI-main/BioID-FaceDatabase-V1.2.zip

In [ ]:
!pip check

ipython 7.34.0 requires jedi, which is not installed.
torch 2.6.0+cu124 has requirement nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2.
torch 2.6.0+cu124 has requirement nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82.
torch 2.6.0+cu124 has requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82.
torch 2.6.0+cu124 has requirement nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82.
torch 2.6.0+cu124 has requirement nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75.
torch 2.6.0+cu124 has requirement nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux

In [2]:
!pip install -q dlib opencv-python

!wget -O predictor.bz2 http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk predictor.bz2

import os
os.rename("predictor", "shape_predictor_68_face_landmarks.dat")

# data.zip
from google.colab import files
uploaded = files.upload()

--2025-06-10 07:27:26--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-06-10 07:27:27--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘predictor.bz2’

predictor.bz2       100%[===================>]  61.07M  36.3MB/s    in 1.7s    

2025-06-10 07:27:29 (36.3 MB/s) - ‘predictor.bz2’ saved [64040097/64040097]



Saving data.zip to data.zip


In [3]:
import zipfile, os

zip_path = next(iter(uploaded))
extract_dir = "data"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

import cv2
import dlib
import numpy as np

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

output_dir = "output_384x286"
os.makedirs(output_dir, exist_ok=True)

from glob import glob
from tqdm import tqdm

img_paths = sorted(glob(f"{extract_dir}/**/*.jpg", recursive=True))

for path in tqdm(img_paths):
    name = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path)
    if img is None:
        continue

    resized = cv2.resize(img, (384, 286))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    dets = detector(gray, 1)
    if len(dets) == 0:
        continue

    shape = predictor(gray, dets[0])
    landmarks = np.array([[p.x, p.y] for p in shape.parts()])

    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]

    left_center = np.mean(left_eye, axis=0)
    right_center = np.mean(right_eye, axis=0)

    cv2.imwrite(f"{output_dir}/{name}.jpg", gray)
    with open(f"{output_dir}/{name}.eye", "w") as f:
        f.write(f"{left_center[0]:.2f} {left_center[1]:.2f} {right_center[0]:.2f} {right_center[1]:.2f}\n")

!zip -r output_384x286.zip output_384x286
files.download("output_384x286.zip")

KeyboardInterrupt: 

In [ ]:
import os, shutil, cv2
from glob import glob

scut_dir = "output_384x286"
bioid_dir = "/content/drive/MyDrive/BioID-FaceDatabase-V1.2"
merged_dir = "BioID-multi-race"

os.makedirs(merged_dir, exist_ok=True)

bioid_pgms = glob(f"{bioid_dir}/**/*.pgm", recursive=True)
bioid_eyes = glob(f"{bioid_dir}/**/*.eye", recursive=True)

for f in bioid_pgms + bioid_eyes:
    shutil.copy(f, os.path.join(merged_dir, os.path.basename(f)))

scut_jpgs = glob(f"{scut_dir}/*.jpg")

for jpg_path in scut_jpgs:
    base = os.path.splitext(os.path.basename(jpg_path))[0]

    img = cv2.imread(jpg_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    pgm_out = os.path.join(merged_dir, base + ".pgm")
    cv2.imwrite(pgm_out, img)

    eye_src = os.path.join(scut_dir, base + ".eye")
    eye_dst = os.path.join(merged_dir, base + ".eye")
    if os.path.exists(eye_src):
        shutil.copy(eye_src, eye_dst)

print(f"Merged: {len(os.listdir(merged_dir))} files in '{merged_dir}/'")

In [ ]:
import shutil
import os

drive_target_dir = "/content/drive/MyDrive/BioID-multi-race"

if os.path.exists(drive_target_dir):
    shutil.rmtree(drive_target_dir)

shutil.copytree("BioID-multi-race", drive_target_dir)

print("EndSave")

In [4]:
!pip install -q opencv-python dlib

import os, cv2, struct, zipfile
import numpy as np
import dlib
from glob import glob
from tqdm import tqdm
from google.colab import files


# data.zip
uploaded = files.upload()

KeyboardInterrupt: 

In [8]:
zip_filename = next(iter(uploaded))

extract_dir = "data"
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

def read_scaled_pts(path, img_shape, target_shape=(384, 286)):
    with open(path, 'rb') as f:
        raw = f.read()
    pts = struct.unpack('f' * 172, raw)
    pts = np.array(pts).reshape((86, 2))
    h0, w0 = img_shape
    w1, h1 = target_shape
    scale_x, scale_y = w1 / w0, h1 / h0
    return pts * [scale_x, scale_y]

out_dir = "output"
os.makedirs(out_dir, exist_ok=True)

img_paths = sorted(glob(f"{extract_dir}/*.jpg"))
for img_path in tqdm(img_paths):
    name = os.path.splitext(os.path.basename(img_path))[0]
    pts_path = os.path.join(extract_dir, name + ".pts")
    if not os.path.exists(pts_path):
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue
    orig_h, orig_w = img.shape[:2]
    resized = cv2.resize(img, (384, 286))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    pts = read_scaled_pts(pts_path, (orig_h, orig_w))
    left_eye = pts[36:42]
    right_eye = pts[42:48]
    left_center = np.mean(left_eye, axis=0)
    right_center = np.mean(right_eye, axis=0)

    y_diff = left_center[1] - right_center[1]
    if abs(y_diff) > 15:
        dets = detector(gray, 1)
        if len(dets) > 0:
            shape = predictor(gray, dets[0])
            landmarks = np.array([[p.x, p.y] for p in shape.parts()])
            left_center = np.mean(landmarks[36:42], axis=0)
            right_center = np.mean(landmarks[42:48], axis=0)

    with open(os.path.join(out_dir, name + ".eye"), "w") as f:
        f.write(f"{left_center[0]:.2f} {left_center[1]:.2f} {right_center[0]:.2f} {right_center[1]:.2f}\n")

    pgm_path = os.path.join(out_dir, name + ".pgm")
    cv2.imwrite(pgm_path, gray)

0it [00:00, ?it/s]


In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [18]:
# # base_path = "/content/drive/MyDrive/SafeAI/BioID-FaceDatabase-V1.2"
# base_path = "/content/drive/MyDrive/BioID-multi-race"
# image_files = sorted([f for f in os.listdir(base_path) if f.endswith(".pgm")])
# label_files = sorted([f for f in os.listdir(base_path) if f.endswith(".eye")])

# print(len(image_files))
# print(len(label_files))

2945
2945


In [9]:
base_path = "/content/drive/MyDrive/BioID-multi-race"
image_files = sorted([f for f in os.listdir(base_path) if f.endswith(".pgm")])
label_files = sorted([f for f in os.listdir(base_path) if f.endswith(".eye")])

print(len(image_files))
print(len(label_files))

data = []
for img_file, label_file in zip(image_files, label_files):
    img_path = os.path.join(base_path, img_file)
    label_path = os.path.join(base_path, label_file)

    with open(label_path, "r") as f:
        lines = f.readlines()
        if len(lines) == 1:
            lx, ly, rx, ry = map(float, lines[0].strip().split())
        else:
          lx, ly, rx, ry = map(float, lines[1].strip().split())
        # print(f"left: {lx}, {ly} / right: {rx}, {ry}")
        data.append((img_path, [lx, ly, rx, ry]))


2945
2945


In [10]:
len(data)

2945

In [11]:
random.seed(42)
random.shuffle(data)

holdout_data = data[:21]    # 비교용
remaining_data = data[21:]  # 학습용

df = pd.DataFrame(remaining_data, columns=["image_path", "eye_coords"])
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

holdout_df = pd.DataFrame(holdout_data, columns=["image_path", "eye_coords"])

print(len(remaining_data))

2924


In [ ]:
# import shutil

# output_dir = "/content/drive/MyDrive/SafeAI/comparisons"
# if os.path.exists(output_dir) == False:
#   os.makedirs(output_dir, exist_ok=True)

#   for img_path in holdout_df["image_path"]:
#       base_name = os.path.basename(img_path)
#       eye_name = base_name.replace(".pgm", ".eye")
#       eye_path = os.path.join(os.path.dirname(img_path), eye_name)

#       dst_img_path = os.path.join(output_dir, base_name)
#       dst_eye_path = os.path.join(output_dir, eye_name)

#       shutil.copy(img_path, dst_img_path)

#       if os.path.exists(eye_path):
#           shutil.copy(eye_path, dst_eye_path)

In [ ]:
# src_dir = "/content/drive/MyDrive/SafeAI/BioID-FaceDatabase-V1.2"
# dst_dir = "/content/drive/MyDrive/SafeAI/comparisons"

# eye_files = ["BioID_1172.eye", "BioID_0331.eye", "BioID_0250.eye"]

# os.makedirs(dst_dir, exist_ok=True)

# for filename in eye_files:
#     src_path = os.path.join(src_dir, filename)
#     dst_path = os.path.join(dst_dir, filename)

#     try:
#         shutil.copy(src_path, dst_path)
#         print(f"복사 완료: {filename}")
#     except Exception as e:
#         print(f"복사 실패: {filename} - {e}")

In [13]:
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=0)  # (1, H, W)
    return image  # (1, 286, 384)

In [14]:
class EyeDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = preprocess_image(row["image_path"])  # (1, 128, 128)
        label = np.array(row["eye_coords"], dtype=np.float32)
        return torch.tensor(image), torch.tensor(label)

In [15]:
class HoldoutDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = preprocess_image(row["image_path"])  # (1, H, W)
        label = np.array(row["eye_coords"], dtype=np.float32)
        return torch.tensor(image), torch.tensor(label)

In [43]:
class EyeCNN(nn.Module):
    def __init__(self):
        super(EyeCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # (286, 384) → (143, 192)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # (143, 192) → (71, 96)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # (71, 96) → (35, 48)

            nn.AdaptiveAvgPool2d((4, 6)), # 128 * 4 * 6 = 3072
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(3072, 256),
            nn.ReLU(),
            nn.Linear(256, 4)  # [lx, ly, rx, ry]
        )

    def forward(self, x):
        return self.model(x)

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [45]:
train_dataset = EyeDataset(train_df)
test_dataset = EyeDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

model = EyeCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 아래부터는 학습

In [46]:
epochs = 100
best_loss = float('inf')
best_model_state = None

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

    if avg_loss < best_loss:
        best_loss = avg_loss
        best_model_state = model.state_dict()


model.load_state_dict(best_model_state)

Epoch [1/100] - Loss: 3652.2826
Epoch [2/100] - Loss: 373.8912
Epoch [3/100] - Loss: 303.7016
Epoch [4/100] - Loss: 281.7010
Epoch [5/100] - Loss: 254.0284
Epoch [6/100] - Loss: 241.9378
Epoch [7/100] - Loss: 231.3098
Epoch [8/100] - Loss: 209.8149
Epoch [9/100] - Loss: 199.2652
Epoch [10/100] - Loss: 194.1430
Epoch [11/100] - Loss: 166.3980
Epoch [12/100] - Loss: 161.1444
Epoch [13/100] - Loss: 156.9005
Epoch [14/100] - Loss: 141.5475
Epoch [15/100] - Loss: 128.3460
Epoch [16/100] - Loss: 127.9983
Epoch [17/100] - Loss: 122.7501
Epoch [18/100] - Loss: 114.0610
Epoch [19/100] - Loss: 109.2723
Epoch [20/100] - Loss: 103.1062
Epoch [21/100] - Loss: 102.7997
Epoch [22/100] - Loss: 102.4666
Epoch [23/100] - Loss: 97.6750
Epoch [24/100] - Loss: 88.4747
Epoch [25/100] - Loss: 91.6818
Epoch [26/100] - Loss: 89.9264
Epoch [27/100] - Loss: 85.2984
Epoch [28/100] - Loss: 85.7816
Epoch [29/100] - Loss: 82.2579
Epoch [30/100] - Loss: 79.5443
Epoch [31/100] - Loss: 79.2179
Epoch [32/100] - Loss: 77

<All keys matched successfully>

In [ ]:
print(best_loss)

In [ ]:
def predict(model, dataset, device):
    model.eval()
    predictions = []

    for i in range(len(dataset)):
        image_tensor, _ = dataset[i]
        image_input = image_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            pred = model(image_input).cpu().numpy()[0]
        predictions.append(pred.tolist())

    return predictions

In [ ]:
def print_cnn_predictions(dataset, predictions, num_samples=21):
    data_pairs = []

    for i in range(min(num_samples, len(dataset))):
        _, label = dataset[i]
        pred = predictions[i]

        gt = label.numpy().tolist()
        pr = np.round(pred, 2).tolist()

        print(f"샘플 {i+1}:")
        print(f" - 정답 좌표 (GT):     {label.numpy().tolist()}")
        print(f" - 예측 좌표 (CNN):    {np.round(pred, 2).tolist()}")
        print()

        data_pairs.append((gt, pr))

    mae_list, mse_list, euclidean_list = [], [], []

    for gt, pr in data_pairs:
        gt = np.array(gt)
        pr = np.array(pr)
        diff = gt - pr

        mae = np.mean(np.abs(diff))
        mse = np.mean(diff ** 2)
        euclidean = np.linalg.norm(diff)

        mae_list.append(mae)
        mse_list.append(mse)
        euclidean_list.append(euclidean)

    print(f"MAE: {np.mean(mae_list):.2f}")
    print(f"MSE: {np.mean(mse_list):.2f}")
    print(f"Euclidean: {np.mean(euclidean_list):.2f}")

In [ ]:
def visualize_predictions(dataset, predictions, num_samples=21, cols=3):
    rows = (num_samples + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))

    for i in range(min(num_samples, len(dataset))):
        img_tensor, label = dataset[i]
        img = img_tensor.numpy()[0]
        img = np.squeeze(img)
        pred = predictions[i]

        ax = axes[i // cols][i % cols] if rows > 1 else axes[i]
        ax.imshow(img, cmap='gray')
        ax.scatter(label[0], label[1], color='blue', label='GT Left')
        ax.scatter(label[2], label[3], color='blue', label='GT Right')
        ax.scatter(pred[0], pred[1], color='red', marker='x', label='Pred Left')
        ax.scatter(pred[2], pred[3], color='red', marker='x', label='Pred Right')
        ax.axis('off')
        if i == 0:
            ax.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
holdout_dataset = HoldoutDataset(holdout_df)

In [ ]:
predictions = predict(model, holdout_dataset, device)

In [ ]:
print_cnn_predictions(holdout_dataset, predictions)

In [ ]:
visualize_predictions(holdout_dataset, predictions, 21, 3)

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/SafeAI_Final/eye_cnn_100epochs.pth")

In [ ]:
# 아래부터 Machine Unlearning

In [ ]:
# 학습한 모델 로드

loaded_model = EyeCNN()
loaded_model.load_state_dict(torch.load("/content/drive/MyDrive/SafeAI_Final/eye_cnn_100epochs.pth"))
loaded_model.eval()

In [ ]:
# 동양인 이미지 찾기

base_path = "/content/drive/MyDrive/BioID-multi-race"
image_files = sorted([f for f in os.listdir(base_path) if f.endswith(".pgm")])

all_asian_indices = []
for idx, img_file in enumerate(image_files):
    if 'AM' in img_file or 'AF' in img_file:
        all_asian_indices.append(idx)

print(f"전체 동양인 이미지 개수: {len(all_asian_indices)}")
print(f"전체 이미지 개수: {len(image_files)}")

asian_sample_indices = []

for train_idx, row in train_df.iterrows():
    image_name = os.path.basename(row["image_path"])
    if 'AM' in image_name or 'AF' in image_name:
        asian_sample_indices.append(train_idx)

print(f"train_df에서 동양인 이미지 개수: {len(asian_sample_indices)}")
print(f"train_df 총 개수: {len(train_df)}")

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_dataset = EyeDataset(train_df)
test_dataset = EyeDataset(test_df)

asian_sample_indices = []
for idx, row in train_df.iterrows():
    image_name = os.path.basename(row["image_path"])
    if 'AM' in image_name or 'AF' in image_name:
        asian_sample_indices.append(idx)

print(f"train_dataset 길이: {len(train_dataset)}")
print(f"asian_sample_indices 최대값: {max(asian_sample_indices) if asian_sample_indices else 'None'}")
print(f"asian_sample_indices 개수: {len(asian_sample_indices)}")

valid_indices = [idx for idx in asian_sample_indices if 0 <= idx < len(train_dataset)]
print(f"유효한 인덱스 개수: {len(valid_indices)}")


In [ ]:
# 삭제하고싶은 동양인 비율



In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import time

In [ ]:
def machine_unlearning_gradient_ascent(model, forget_loader, retain_loader, lr=1e-5, epochs=5):
    """Gradient ascent unlearning with debugging"""
    print(f"[Stage 1] Gradient Ascent Unlearning")
    print(f"Learning rate: {lr}, Epochs: {epochs}")
    print(f"Forget samples: {len(forget_loader.dataset)}, Retain samples: {len(retain_loader.dataset)}")

    for epoch in range(epochs):
        model.train()
        epoch_forget_loss = 0
        epoch_retain_loss = 0

        # Forget phase
        for batch_idx, (data, targets) in enumerate(forget_loader):
            # 🐛 디버깅: 첫 배치 확인
            if epoch == 0 and batch_idx == 0:
                print(f"[DEBUG] Batch 0 - data shape: {data.shape}, targets shape: {targets.shape}")
                print(f"[DEBUG] Data has nan?: {torch.isnan(data).any().item()}, inf?: {torch.isinf(data).any().item()}")
                print(f"[DEBUG] Targets has nan?: {torch.isnan(targets).any().item()}, inf?: {torch.isinf(targets).any().item()}")

                with torch.no_grad():
                    output = model(data)
                    print(f"[DEBUG] Output min: {output.min().item()}, max: {output.max().item()}")
                    print(f"[DEBUG] Output has nan?: {torch.isnan(output).any().item()}, inf?: {torch.isinf(output).any().item()}")

                    try:
                        debug_loss = F.mse_loss(output, targets)
                        print(f"[DEBUG] MSE loss: {debug_loss.item()}")
                    except Exception as e:
                        print(f"[DEBUG] Loss computation error: {e}")

            model.zero_grad()
            predictions = model(data)
            forget_loss = F.mse_loss(predictions, targets)
            forget_loss.backward()

            # Gradient ascent 적용 (optimizer 사용 안함)
            for param in model.parameters():
                if param.grad is not None:
                    param.data += lr * param.grad

            epoch_forget_loss += forget_loss.item()

        # Retain phase (기존 방식 그대로)
        for batch_idx, (data, targets) in enumerate(retain_loader):
            model.zero_grad()
            predictions = model(data)
            retain_loss = F.mse_loss(predictions, targets)
            retain_loss.backward()
            for param in model.parameters():
                if param.grad is not None:
                    param.data -= lr * param.grad  # gradient descent
            epoch_retain_loss += retain_loss.item()

        avg_forget = epoch_forget_loss / len(forget_loader)
        avg_retain = epoch_retain_loss / len(retain_loader)
        print(f"Epoch {epoch+1}: Forget Loss {avg_forget:.6f}, Retain Loss {avg_retain:.6f}")

    print(f"Stage 1 completed\n")
    return model

In [ ]:
def adaptive_unlearning(model, asian_loader, western_loader):
    """Adaptive parameter adjustment with minimal logging"""
    print(f"[Stage 2] Adaptive Parameter Adjustment")

    print("Computing Fisher Information...")
    fisher_asian = compute_fisher_information(model, asian_loader)
    fisher_western = compute_fisher_information(model, western_loader)

    print("Calculating differential importance...")
    differential_importance = {}
    total_params = 0
    adjusted_params = 0

    for name in fisher_asian:
        differential_importance[name] = fisher_asian[name] - fisher_western[name]
        total_params += fisher_asian[name].numel()

    print("Adjusting parameters...")
    for name, param in model.named_parameters():
        if name in differential_importance:
            importance_diff = differential_importance[name]
            adjustment_mask = importance_diff > 0
            adjusted_count = adjustment_mask.sum().item()
            adjusted_params += adjusted_count

            with torch.no_grad():
                param[adjustment_mask] *= 0.1

    print(f"Adjusted {adjusted_params}/{total_params} parameters ({adjusted_params/total_params*100:.1f}%)")
    print(f"Stage 2 completed\n")

    return model


In [ ]:
def compute_fisher_information(model, data_loader):
    """Compute Fisher Information Matrix"""
    fisher_dict = {}
    model.eval()

    for name, param in model.named_parameters():
        fisher_dict[name] = torch.zeros_like(param)

    for batch_idx, (data, targets) in enumerate(data_loader):
        model.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, targets)
        loss.backward()

        for name, param in model.named_parameters():
            if param.grad is not None:
                fisher_dict[name] += param.grad.pow(2)

    for name in fisher_dict:
        fisher_dict[name] /= len(data_loader)

    return fisher_dict


In [ ]:
def neuron_level_unlearning(model, asian_indices, prune_ratio=0.2):
    """Structural pruning with minimal logging"""
    print(f"[Stage 3] Structural Pruning")
    print(f"Pruning ratio: {prune_ratio*100:.1f}%")

    import torch.nn.utils.prune as prune

    pruned_layers = 0
    total_params_before = sum(p.numel() for p in model.parameters())

    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=prune_ratio)
            prune.remove(module, 'weight')
            pruned_layers += 1

    total_params_after = sum(p.numel() for p in model.parameters())
    reduction_rate = (total_params_before-total_params_after)/total_params_before*100

    print(f"Pruned {pruned_layers} layers")
    print(f"Parameters: {total_params_before} -> {total_params_after} ({reduction_rate:.1f}% reduction)")
    print(f"Stage 3 completed\n")

    return model


In [ ]:
def complete_unlearning_pipeline(model, dataset, asian_indices):
    """Complete machine unlearning pipeline with minimal logging"""
    print(f"Machine Unlearning Pipeline Started")
    print(f"Total data: {len(dataset)}, Asian samples: {len(asian_indices)}")
    print(f"Removal ratio: {len(asian_indices)/len(dataset)*100:.1f}%\n")

    # Data splitting
    asian_data = Subset(dataset, asian_indices)
    western_indices = [i for i in range(len(dataset)) if i not in asian_indices]
    western_data = Subset(dataset, western_indices)

    asian_loader = DataLoader(asian_data, batch_size=32, shuffle=True)
    western_loader = DataLoader(western_data, batch_size=32, shuffle=True)

    # Stage 1: Gradient ascent unlearning
    model = machine_unlearning_gradient_ascent(model, asian_loader, western_loader, epochs=5)

    # Stage 2: Adaptive parameter adjustment
    model = adaptive_unlearning(model, asian_loader, western_loader)

    # Stage 3: Structural pruning
    model = neuron_level_unlearning(model, asian_indices)

    print(f"Machine Unlearning Pipeline Completed")
    print(f"Processed {len(asian_indices)} Asian samples")

    return model


In [ ]:
if len(valid_indices) > 0:
    unlearned_model = complete_unlearning_pipeline(
        loaded_model,
        train_dataset,
        valid_indices
    )
    print("Success")
else:
    print("IndexOutofRange")


# 결과 저장
torch.save(unlearned_model.state_dict(), 'eye_cnn_unlearned.pth')
print("Model Saved: 'eye_cnn_unlearned.pth'")


In [ ]:
# 테스트 데이터에서 동양인/서양인 인덱스 추출
test_asian_indices = []
test_western_indices = []

for idx, row in test_df.iterrows():
    image_name = os.path.basename(row["image_path"])
    if 'AM' in image_name or 'AF' in image_name:
        test_asian_indices.append(idx)
    else:
        test_western_indices.append(idx)

print(f"테스트 동양인 이미지: {len(test_asian_indices)}개")
print(f"테스트 서양인 이미지: {len(test_western_indices)}개")


In [ ]:
from torch.utils.data import Subset, DataLoader

# 동양인/서양인 테스트 데이터셋 생성
test_asian_dataset = Subset(test_dataset, test_asian_indices)
test_western_dataset = Subset(test_dataset, test_western_indices)

# 데이터로더 생성
test_asian_loader = DataLoader(test_asian_dataset, batch_size=32, shuffle=False)
test_western_loader = DataLoader(test_western_dataset, batch_size=32, shuffle=False)


In [ ]:
# 원본 모델 로드 (언러닝 전)
original_model = torch.load('eye_cnn_100epochs.pth')

# 언러닝 효과 평가 실행
unlearning_ratio, retention_ratio = evaluate_unlearning_effectiveness(
    original_model=original_model,
    unlearned_model=unlearned_model,
    test_asian_loader=test_asian_loader,
    test_western_loader=test_western_loader
)

print(f"\n=== 머신 언러닝 효과 분석 ===")
print(f"동양인 데이터 언러닝 비율: {unlearning_ratio:.4f}")
print(f"서양인 데이터 유지 비율: {retention_ratio:.4f}")


In [ ]:
# 결과 해석 함수
def interpret_results(unlearning_ratio, retention_ratio):
    print(f"\n=== 결과 해석 ===")

    if unlearning_ratio > 1.2:
        print("✅ 동양인 데이터 언러닝 효과 우수 (20% 이상 성능 저하)")
    elif unlearning_ratio > 1.0:
        print("⚠️ 동양인 데이터 언러닝 효과 보통")
    else:
        print("❌ 동양인 데이터 언러닝 효과 미흡")

    if 0.9 <= retention_ratio <= 1.1:
        print("✅ 서양인 데이터 성능 잘 유지됨")
    else:
        print("⚠️ 서양인 데이터 성능에 영향 있음")

# 결과 해석
interpret_results(unlearning_ratio, retention_ratio)


In [ ]:
import cv2
import os
import numpy as np

def draw_black_boxes_from_predictions(predictions, holdout_df, output_dir, box_height=20):
    os.makedirs(output_dir, exist_ok=True)

    for i, pred in enumerate(predictions):
        try:
            lx, ly, rx, ry = map(int, pred)
            image_path = holdout_df.iloc[i]["image_path"]
            filename = os.path.basename(image_path)
            filename_no_ext = os.path.splitext(filename)[0]
            output_path = os.path.join(output_dir, f"boxed_{filename_no_ext}.jpg")

            image = cv2.imread(image_path)

            x_min = max(min(lx, rx) - 20, 0)
            x_max = min(max(lx, rx) + 20, image.shape[1] - 1)

            y_center = (ly + ry) // 2
            y_min = max(y_center - box_height // 2, 0)
            y_max = min(y_center + box_height // 2, image.shape[0] - 1)

            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 0), thickness=-1)

            result = cv2.imwrite(output_path, image)
            if result:
                print(f"저장 완료: {output_path}")
            else:
                print(f"저장 실패: {output_path}")

        except Exception as e:
            print(f"Error [{i}]: {e}")

In [ ]:
predictions = predict(model, holdout_dataset, device)

draw_black_boxes_from_predictions(
    predictions=predictions,
    holdout_df=holdout_df,
    output_dir="/content/drive/MyDrive/SafeAI/boxed_holdout",
    box_height=20
)

저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_1181.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0134.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_1245.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_1115.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0937.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0293.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0921.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_1501.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0182.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0181.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0989.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0729.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0213.jpg
저장 완료: /content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0

In [ ]:
from matplotlib import pyplot as plt
img = cv2.imread("/content/drive/MyDrive/SafeAI/boxed_holdout/boxed_BioID_0134.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [29]:
torch.save(model.state_dict(), "/content/drive/MyDrive/SafeAI_Final/eye_cnn_100epochs.pth")

In [ ]:
# 불러오기

# model = EyeCNN()
# model.load_state_dict(torch.load("eye_cnn.pth"))
# model.eval()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

### RetinaFace

In [ ]:
!pip install retina-face opencv-python-headless

In [ ]:
import os
import cv2
import numpy as np
from retinaface import RetinaFace

def compare_eye_coords(input_dir):
    files = sorted([f for f in os.listdir(input_dir) if f.lower().endswith(".pgm")])
    success_count = 0
    fail_count = 0

    mae_list, mse_list, euclidean_list = [], [], []
    data_pairs = []

    def read_eye_file(eye_path):
        try:
            with open(eye_path, 'r') as f:
                lines = f.readlines()
                if len(lines) < 2:
                    return None
                values = lines[1].strip().split()
                if len(values) != 4:
                    return None
                lx, ly, rx, ry = map(int, values)
                return (lx, ly), (rx, ry)
        except Exception as e:
            print(f"file error: {eye_path} - {e}")
            return None

    print("filename\tGT_L\tGT_R\tPR_L\tPR_R\tL_error\tR_error")

    for filename in files:
        image_path = os.path.join(input_dir, filename)
        eye_path = image_path.replace(".pgm", ".eye")

        gt_coords = read_eye_file(eye_path)
        gt_left, gt_right = gt_coords

        gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

        try:
            faces = RetinaFace.detect_faces(rgb)
            if not faces:
                print(f"No face: {filename}")
                fail_count += 1
                continue

            face = list(faces.values())[0]
            landmarks = face["landmarks"]
            pr_left = landmarks["left_eye"]
            pr_right = landmarks["right_eye"]

            l_error = round(np.linalg.norm(np.array(gt_left) - np.array(pr_left)), 2)
            r_error = round(np.linalg.norm(np.array(gt_right) - np.array(pr_right)), 2)

            print(f"{filename}\t{gt_left}\t{gt_right}\t{pr_left}\t{pr_right}\t{l_error}\t{r_error}")
            success_count += 1

            # 전체 지표를 위한 누적
            gt = np.array([*gt_left, *gt_right])
            pr = np.array([*pr_left, *pr_right])
            data_pairs.append((gt, pr))

        except Exception as e:
            print(f"Exception Error: {filename} - {type(e).__name__}: {e}")
            fail_count += 1

    # 📊 SOTA 지표 계산
    for gt, pr in data_pairs:
        diff = gt - pr
        mae = np.mean(np.abs(diff))
        mse = np.mean(diff ** 2)
        euclidean = np.linalg.norm(diff)

        mae_list.append(mae)
        mse_list.append(mse)
        euclidean_list.append(euclidean)

    print(f"\nMAE: {np.mean(mae_list):.2f}")
    print(f"MSE: {np.mean(mse_list):.2f}")
    print(f"Euclidean: {np.mean(euclidean_list):.2f}")

In [ ]:
compare_eye_coords("/content/drive/MyDrive/SafeAI/comparisons")

filename	GT_L	GT_R	PR_L	PR_R	L_error	R_error
BioID_0140.pgm	(243, 119)	(179, 116)	[np.float32(239.53946), np.float32(114.65381)]	[np.float32(178.13518), np.float32(114.47114)]	5.56	1.76
BioID_0207.pgm	(242, 113)	(177, 112)	[np.float32(241.50998), np.float32(110.82748)]	[np.float32(179.08981), np.float32(111.001785)]	2.23	2.32
BioID_0211.pgm	(232, 115)	(167, 112)	[np.float32(229.87245), np.float32(112.67509)]	[np.float32(167.9954), np.float32(109.987915)]	3.15	2.24
BioID_0250.pgm	(259, 113)	(208, 111)	[np.float32(260.0051), np.float32(112.56578)]	[np.float32(208.38177), np.float32(109.92682)]	1.09	1.14
BioID_0331.pgm	(204, 145)	(160, 142)	[np.float32(201.7934), np.float32(144.59041)]	[np.float32(157.7959), np.float32(142.94089)]	2.24	2.4
BioID_0400.pgm	(164, 100)	(119, 98)	[np.float32(162.13423), np.float32(100.43073)]	[np.float32(117.47988), np.float32(99.757996)]	1.91	2.32
BioID_0446.pgm	(179, 100)	(119, 99)	[np.float32(180.51622), np.float32(101.72968)]	[np.float32(117.768814), np.fl